# 基于百度 AI 的情绪标记

In [ ]:
import pymysql

In [1]:
import pandas as pd  # 导入必要的库
import requests  # 导入必要的库
import json  # 导入必要的库
import time  # 导入必要的库

In [2]:
# !pip install baidu-aip

In [3]:
# 导入百度API的AipNlp类以使用百度的自然语言处理功能
from aip import AipNlp  # 导入必要的库

In [4]:
# 以下三个值需要替换为你的应用信息，使用时候改为自己的即可
APP_ID = '---'
API_KEY = '---'
SECRET_KEY = '---'

# 创建AipNlp实例,用于后续使用
aipNlp = AipNlp(APP_ID, API_KEY, SECRET_KEY)  # 实例化AipNlp类，使用你的百度云应用的APP_ID, API_KEY和SECRET_KEY

In [ ]:
# 连接数据库
def connect_to_database(host, user, password, database, retries=5, delay=5):
    connection = None
    while not connection and retries > 0:
        try:
            connection = pymysql.connect(host=host,
                                         user=user,
                                         password=password,
                                         database=database,
                                         charset='utf8mb4',
                                         cursorclass=pymysql.cursors.DictCursor)
            print("已成功连接到数据库")
        except Exception as e:
            print(f"连接失败：{e}")
            retries -= 1
            print(f"重试连接，剩余尝试次数：{retries}")
            if retries > 0:
                time.sleep(delay)
    return connection

In [ ]:
# 向数据库插入数据
def insert_data(connection, table_name, data):
    with connection.cursor() as cursor:
        # Insert data into temporary table
        keys = ', '.join(data.keys())
        values = ', '.join(['%s'] * len(data))
        sql = f"INSERT INTO {table_name} ({keys}) VALUES ({values})"
        cursor.execute(sql, tuple(data.values()))
    connection.commit()

In [65]:
# 定义一个情感分析函数，通过调用百度API评估评论的情绪倾向，并在每次API调用后暂停一定时间来避免频率限制
def posi_or_nega(comment, sleeping):
time.sleep(sleeping)  # 等待一定时间，避免频繁请求API导致访问限制
try:  # 异常处理，尝试执行代码块，处理潜在的错误
posi_prob = (aipNlp.sentimentClassify(comment))['items'][0]['positive_prob']  # 条件判断
nega_prob = (aipNlp.sentimentClassify(comment))['items'][0]['negative_prob']  # 条件判断
if posi_prob > 0.7:  # 条件判断
return 1
elif posi_prob < 0.3:  # 条件判断
return (-1)
else:  # 条件判断
return 0
except:  # 异常处理，尝试执行代码块，处理潜在的错误
try:  # 异常处理，尝试执行代码块，处理潜在的错误
error = (aipNlp.sentimentClassify(comment))['error_msg']  # 条件判断
return error
except:  # 异常处理，尝试执行代码块，处理潜在的错误
return 'connection loss'

In [6]:
# 定义一个函数来检查是否有因连接问题而未成功返回结果的请求，该函数输出失败的请求数量及其索引
def check_loss(sentiment_col):
loss_num = sentiment_col[sentiment_col == 'connection loss'].count()
total_num = len(sentiment_col)
loss_index = list(sentiment_col[sentiment_col == 'connection loss'].index)
check_result = {
'loss_number': loss_num,
'loss_ratio': loss_num/total_num,
'total_number': total_num,
'loss_index': loss_index
}
return check_result

#### 一个 sample 示范

In [99]:
# 从CSV文件导入评论数据，并展示数据的前几行以检查其结构
comments_df = pd.read_csv('/Users/wujiewang/Desktop/QF5214/stock_comments.csv')  # 使用pandas的read_csv函数读取CSV文件为DataFrame
comments_df.head()

,stock_code,time,comments,user_name
0,1,03-29 07:57,平安银行每股派0.719元，股息率6.9%。兴业银行每股派1.040元，股息率6,海兴大吉
1,1,03-27 08:38,平安银行目前股息率最高,价值投资棒棒棒
2,1,03-26 07:13,【炜炜道来】聊聊本轮调整时的操作策略和风险点,边风炜说投资
3,1,03-24 09:08,部分银行年报业绩快报点评,价值投资棒棒棒
4,1,03-23 07:15,人们纠结的还是分红，前几天由于平安银行的正面示范作用，长沙走得还挺好，昨天，中信,轻松的苗飞文


In [100]:
# 选择数据的一个子集作为示例，用于演示后续的情感分析处理
session = comments_df.iloc[:30, :]

In [101]:
# 对选定的评论数据执行情感分析，将结果保存在新的列'sentiments'中
session['sentiments'] = [posi_or_nega(comment, 0.5) for comment in session['comments']]

/var/folders/lw/5lp1wqys1c50w56nj8g_n_7w0000gn/T/ipykernel_19502/1085485477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session['sentiments'] = [posi_or_nega(comment, 0.5) for comment in session['comments']]


In [102]:
# 调用前面定义的check_loss函数，检查情感分析结果中的失败请求情况
check_loss(session['sentiments'])

{'loss_number': 3,
 'loss_ratio': 0.1,
 'total_number': 30,
 'loss_index': [3, 12, 20]}

In [103]:
session

,stock_code,time,comments,user_name,sentiments
0,1,03-29 07:57,平安银行每股派0.719元，股息率6.9%。兴业银行每股派1.040元，股息率6,海兴大吉,1
1,1,03-27 08:38,平安银行目前股息率最高,价值投资棒棒棒,1
2,1,03-26 07:13,【炜炜道来】聊聊本轮调整时的操作策略和风险点,边风炜说投资,1
3,1,03-24 09:08,部分银行年报业绩快报点评,价值投资棒棒棒,connection loss
4,1,03-23 07:15,人们纠结的还是分红，前几天由于平安银行的正面示范作用，长沙走得还挺好，昨天，中信,轻松的苗飞文,1
5,1,03-23 03:21,说说如何找到正确的思路吧,随便玩玩77,-1
6,1,03-21 08:31,平安银行2023减收增利，拿30%净利分红！全靠“勒紧裤腰带”？|年报解码,科记汇,-1
7,1,03-21 03:30,年报密集发布，上市企业普遍提高分红，我们该如何辨别真伪呢,投资观2021,-1
8,1,03-20 03:39,有人说资金少买3家公司好了，但是我有自己的想法，记得刚买康美才一个月的时候就爆掉,琳达linda,1
9,1,03-19 09:38,伯克希尔公司历来不支付股息，而股价随时间的延伸持续保持上升，由每股几美元、十几美,ZJZXTNJNQLCFGJHX,1


In [106]:
session.to_csv('/Users/wujiewang/Desktop/QF5214/sample_sentimentClassify.csv', encoding = 'utf_8_sig')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文

## 一个应用：对沪深 300 股指成分股提取的新闻标题进行情感判断

In [9]:
holding_stock = pd.read_csv('/Users/wujiewang/Desktop/QF5214/stock_news_titles_like_comments.csv')  # 使用pandas的read_csv函数读取CSV文件为DataFrame
holding_stock

,stock_code,release_time,news_titles,reading_numb,comment_numb,author
0,1,04-01 08:51,平安银行：融资净偿还9308.54万元，两市排名第15（03-29）,98,0,平安银行资讯
1,1,03-31 03:07,平安银行本周融资净偿还1.71亿元，两市排名第13,137,0,大眼看两融
2,1,03-31 02:11,平安银行本周深股通持股市值增加1.89亿元，居银行板块第五,191,1,胖猫周周见
3,1,03-29 04:54,平安银行：截至2023年末 本行共有在职员工43119人（含派遣人员、含理财子和分行）,2802,10,平安银行资讯
4,1,03-29 02:48,平安发布《2023年可持续发展报告》 ESG评级保持国内领先,896,3,平安银行资讯
...,...,...,...,...,...,...
71995,688981,11-14 01:57,光大证券维持中芯国际买入评级 预计2023年净利润同比减少50.33%,192,0,公司研报提示
71996,688981,11-14 07:45,中芯国际：连续6日融资净买入累计3.21亿元（11-13）,221,0,中芯国际资讯
71997,688981,11-14 07:44,华金证券给予中芯国际买入评级，代工龙头营收持续增长，设备提前进场加速产能扩张,2749,19,中芯国际资讯
71998,688981,11-14 07:04,【调研快报】中芯国际接待华泰证券等多家机构调研,1870,1,中芯国际资讯


In [10]:
holding_stock['stock_code'] = holding_stock['stock_code'].astype('str')
holding_stock['stock_code'] = holding_stock['stock_code'].str.zfill(6)
holding_stock

,stock_code,release_time,news_titles,reading_numb,comment_numb,author
0,000001,04-01 08:51,平安银行：融资净偿还9308.54万元，两市排名第15（03-29）,98,0,平安银行资讯
1,000001,03-31 03:07,平安银行本周融资净偿还1.71亿元，两市排名第13,137,0,大眼看两融
2,000001,03-31 02:11,平安银行本周深股通持股市值增加1.89亿元，居银行板块第五,191,1,胖猫周周见
3,000001,03-29 04:54,平安银行：截至2023年末 本行共有在职员工43119人（含派遣人员、含理财子和分行）,2802,10,平安银行资讯
4,000001,03-29 02:48,平安发布《2023年可持续发展报告》 ESG评级保持国内领先,896,3,平安银行资讯
...,...,...,...,...,...,...
71995,688981,11-14 01:57,光大证券维持中芯国际买入评级 预计2023年净利润同比减少50.33%,192,0,公司研报提示
71996,688981,11-14 07:45,中芯国际：连续6日融资净买入累计3.21亿元（11-13）,221,0,中芯国际资讯
71997,688981,11-14 07:44,华金证券给予中芯国际买入评级，代工龙头营收持续增长，设备提前进场加速产能扩张,2749,19,中芯国际资讯
71998,688981,11-14 07:04,【调研快报】中芯国际接待华泰证券等多家机构调研,1870,1,中芯国际资讯


In [20]:
new_titles = holding_stock['news_titles']
new_titles

0                 平安银行：融资净偿还9308.54万元，两市排名第15（03-29）
1                          平安银行本周融资净偿还1.71亿元，两市排名第13
2                      平安银行本周深股通持股市值增加1.89亿元，居银行板块第五
3        平安银行：截至2023年末 本行共有在职员工43119人（含派遣人员、含理财子和分行）
4                     平安发布《2023年可持续发展报告》 ESG评级保持国内领先
                            ...                     
71995            光大证券维持中芯国际买入评级 预计2023年净利润同比减少50.33%
71996                  中芯国际：连续6日融资净买入累计3.21亿元（11-13）
71997         华金证券给予中芯国际买入评级，代工龙头营收持续增长，设备提前进场加速产能扩张
71998                        【调研快报】中芯国际接待华泰证券等多家机构调研
71999                      中芯国际：预见大宗产品晶圆价格在来年还会进一步下降
Name: news_titles, Length: 72000, dtype: object

In [21]:
list(range(0, 20, 10))

[0, 10]

In [17]:
session_container = []
for i in range(0, 20, 10):
session = holding_stock
classify_part_j = [ posi_or_nega(comment, 1) for comment in [i:i+10] ]  # 条件判断
session_container.extend(classify_part_j)  # 条件判断
sentiment_classify = pd.DataFrame(session_container)  # 条件判断

In [44]:
import warnings  # 导入必要的库
warnings.filterwarnings('ignore')

In [42]:
session_container = []
j = 0
for i in range(0, 20, 10):
j += 1
session = holding_stock.iloc[i:i+10, :]
classify_part_j = [ posi_or_nega(comment, 1) for comment in new_titles[i:i+10] ]  # 条件判断
session['sentiment'] = classify_part_j  # 条件判断
session.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/sentiment_classify_{j}.csv', encoding = 'utf_8_sig')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
print(f'表{j}输出成功!')
sentiment_classify = pd.DataFrame(session_container)  # 条件判断

/var/folders/lw/5lp1wqys1c50w56nj8g_n_7w0000gn/T/ipykernel_38933/865477639.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session['sentiment'] = classify_part_j


表1输出成功!
表2输出成功!


/var/folders/lw/5lp1wqys1c50w56nj8g_n_7w0000gn/T/ipykernel_38933/865477639.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session['sentiment'] = classify_part_j


In [50]:
j = 0
for i in range(0, len(holding_stock), 5000):
j += 1
session = holding_stock.iloc[i:i+5000, :]
classify_part_j = [ posi_or_nega(comment, 0.5) for comment in new_titles[i:i+5000] ]  # 条件判断
session['sentiment'] = classify_part_j  # 条件判断
session.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/sentiment_classify_{j}.csv', encoding = 'utf_8_sig')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
print(f'表{j}输出成功!')

表1输出成功!
表2输出成功!
表3输出成功!
表4输出成功!
表5输出成功!
表6输出成功!
表7输出成功!
表8输出成功!
表9输出成功!
表10输出成功!
表11输出成功!
表12输出成功!
表13输出成功!
表14输出成功!
表15输出成功!


In [56]:
# 打开 csv，合并到一张表。
container = []
for i in range(1, 16):
table = pd.read_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/sentiment_classify_{i}.csv', index_col = 0)  # 使用pandas的read_csv函数读取CSV文件为DataFrame
container.append(table)
result = pd.concat(container, ignore_index = True)

In [57]:
result[result['sentiment'] == 'connection loss'

0        False
1        False
2        False
3        False
4         True
         ...  
71995    False
71996     True
71997    False
71998    False
71999    False
Name: sentiment, Length: 72000, dtype: bool

In [64]:
loss_chunk['news_titles']

4                平安发布《2023年可持续发展报告》 ESG评级保持国内领先
12            平安银行：融资净偿还7821.12万元，两市排名第九（03-25）
18                    平安银行03月22日被深股通减持1225.33万股
25        平安银行：融资净偿还177.82万元，融资余额53.73亿元（03-19）
30               压缩高风险业务、分行回归经营主战场，平安银行新思路的变与不变
                          ...                  
71976    中芯国际：融资净偿还3635.35万元，融资余额37.04亿元（11-21）
71981          中芯国际本周被沪股通减持2.53万股 最新持股量544.79万股
71986                    中芯国际11月15日被沪股通减持8.68万股
71991                    晶圆产能过剩预警 A股芯片产业链来年静待花开
71996             中芯国际：连续6日融资净买入累计3.21亿元（11-13）
Name: news_titles, Length: 12398, dtype: object

In [ ]:
check_loss(result['sentiment'])

In [66]:
loss_chunk = result[result['sentiment'] == 'connection loss']

In [67]:
loss_chunk

,stock_code,release_time,news_titles,reading_numb,comment_numb,author,sentiment
4,1,03-29 02:48,平安发布《2023年可持续发展报告》 ESG评级保持国内领先,896,3,平安银行资讯,connection loss
12,1,03-26 08:45,平安银行：融资净偿还7821.12万元，两市排名第九（03-25）,149,0,平安银行资讯,connection loss
18,1,03-23 03:49,平安银行03月22日被深股通减持1225.33万股,362,0,平安银行资讯,connection loss
25,1,03-20 08:53,平安银行：融资净偿还177.82万元，融资余额53.73亿元（03-19）,167,0,平安银行资讯,connection loss
30,1,03-18 10:10,压缩高风险业务、分行回归经营主战场，平安银行新思路的变与不变,1715,2,平安银行资讯,connection loss
...,...,...,...,...,...,...,...
71976,688981,11-22 07:44,中芯国际：融资净偿还3635.35万元，融资余额37.04亿元（11-21）,257,1,中芯国际资讯,connection loss
71981,688981,11-19 02:46,中芯国际本周被沪股通减持2.53万股 最新持股量544.79万股,304,1,胖猫周周见,connection loss
71986,688981,11-16 10:27,中芯国际11月15日被沪股通减持8.68万股,188,0,中芯国际资讯,connection loss
71991,688981,11-14 08:41,晶圆产能过剩预警 A股芯片产业链来年静待花开,4385,8,中芯国际资讯,connection loss


In [71]:
from datetime import datetime  # 导入必要的库

# 获取当前时间
now = datetime.now()

# 打印当前时间
print("当前时间:", now)

当前时间: 2024-04-05 12:48:03.579033


In [76]:
############## try once
# 提取出连接失败的行
loss_chunk = result[result['sentiment'] == 'connection loss'].iloc[:20, :]
# loss_chunk.to_csv('/Users/wujiewang/Desktop/QF5214/sentiment_output/loss_chunk.csv')
# 提取出判断成功的行
#succeed_chunk = result[~(result['sentiment'] == 'connection loss')]
#succeed_chunk.to_csv('/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_chunk.csv')
# 对连接失败行进行重新检测判断(依旧用同一种方法，不管怎么样都会更快)
j = 0
for i in range(4):
if j < 3:  # 条件判断
print(f'第{j+1}次开始时间:', datetime.now()) # 打印出运行时间
loss_chunk['sentiment'] = loss_chunk['news_titles'].apply(lambda x: posi_or_nega(x, 0.5))
# 取增量 succeed chunk
succeed = loss_chunk[~(loss_chunk['sentiment'] == 'connection loss')]
succeed.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_{j+1}.csv') # 第一次处理后弥补的信息  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
# 取新的 loss chunk
loss_chunk = loss_chunk[loss_chunk['sentiment'] == 'connection loss']
loss_chunk.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/loss_{j+1}.csv') #第j+1次处理后仍然剩下的loss  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
print(f'第{j+1}次重新标记成功！结束时间为：', datetime.now())
else:  # 条件判断
print('第4次开始时间:', datetime.now()) # 打印出运行时间
loss_chunk['sentiment'] = loss_chunk['news_titles'].apply(lambda x: posi_or_nega(x, 1))
succeed = loss_chunk[~(loss_chunk['sentiment'] == 'connection loss')]
succeed.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_{4}.csv')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
loss_chunk = loss_chunk[(loss_chunk['sentiment'] == 'connection loss')]
loss_chunk.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/final_loss.csv')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
print('第4次重新标记成功！结束时间为：', datetime.now()) # 最后拼接的，应该是所有 succeed表格，然后拼接上最后的 final loss
j += 1
# 输出再次判断后结果，查看缺失
print(check_loss(loss_chunk['sentiment']))

第1次开始时间: 2024-04-05 13:05:07.144810
第1次重新标记成功！结束时间为： 2024-04-05 13:05:27.181697
第2次开始时间: 2024-04-05 13:05:27.181880
第2次重新标记成功！结束时间为： 2024-04-05 13:05:29.138268
第3次开始时间: 2024-04-05 13:05:29.138488
第3次重新标记成功！结束时间为： 2024-04-05 13:05:30.213510
第4次开始时间: 2024-04-05 13:05:30.213667
第4次重新标记成功！结束时间为： 2024-04-05 13:05:31.607800
{'loss_number': 0, 'loss_ratio': nan, 'total_number': 0, 'loss_index': []}


In [77]:
############## try all
# 提取出连接失败的行
loss_chunk = result[result['sentiment'] == 'connection loss']
# loss_chunk.to_csv('/Users/wujiewang/Desktop/QF5214/sentiment_output/loss_chunk.csv')
# 提取出判断成功的行
#succeed_chunk = result[~(result['sentiment'] == 'connection loss')]
#succeed_chunk.to_csv('/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_chunk.csv')
# 对连接失败行进行重新检测判断(依旧用同一种方法，不管怎么样都会更快)
j = 0
for i in range(4):
if j < 3:  # 条件判断
print(f'第{j+1}次开始时间:', datetime.now()) # 打印出运行时间
loss_chunk['sentiment'] = loss_chunk['news_titles'].apply(lambda x: posi_or_nega(x, 0.5))
# 取增量 succeed chunk
succeed = loss_chunk[~(loss_chunk['sentiment'] == 'connection loss')]
succeed.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_{j+1}.csv') # 第一次处理后弥补的信息  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
# 取新的 loss chunk
loss_chunk = loss_chunk[loss_chunk['sentiment'] == 'connection loss']
loss_chunk.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/loss_{j+1}.csv') #第j+1次处理后仍然剩下的loss  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
print(f'第{j+1}次重新标记成功！结束时间为：', datetime.now())
else:  # 条件判断
print('第4次开始时间:', datetime.now()) # 打印出运行时间
loss_chunk['sentiment'] = loss_chunk['news_titles'].apply(lambda x: posi_or_nega(x, 1))
succeed = loss_chunk[~(loss_chunk['sentiment'] == 'connection loss')]
succeed.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_{4}.csv')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
loss_chunk = loss_chunk[(loss_chunk['sentiment'] == 'connection loss')]
loss_chunk.to_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/final_loss.csv')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文
print('第4次重新标记成功！结束时间为：', datetime.now()) # 最后拼接的，应该是所有 succeed表格，然后拼接上最后的 final loss
j += 1
# 输出再次判断后结果，查看缺失
print(check_loss(loss_chunk['sentiment']))

第1次开始时间: 2024-04-05 13:10:52.583904
第1次重新标记成功！结束时间为： 2024-04-05 16:21:26.522343
第2次开始时间: 2024-04-05 16:21:26.525672
第2次重新标记成功！结束时间为： 2024-04-05 17:05:26.159871
第3次开始时间: 2024-04-05 17:05:26.163019
第3次重新标记成功！结束时间为： 2024-04-05 17:15:53.407198
第4次开始时间: 2024-04-05 17:15:53.407932
第4次重新标记成功！结束时间为： 2024-04-05 17:19:29.569471
{'loss_number': 0, 'loss_ratio': nan, 'total_number': 0, 'loss_index': []}


In [79]:
succeed_0 = pd.read_csv('/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_chunk.csv', index_col = 0)  # 使用pandas的read_csv函数读取CSV文件为DataFrame
succeed_0

,stock_code,release_time,news_titles,reading_numb,comment_numb,author,sentiment
0,1,04-01 08:51,平安银行：融资净偿还9308.54万元，两市排名第15（03-29）,98,0,平安银行资讯,1
1,1,03-31 03:07,平安银行本周融资净偿还1.71亿元，两市排名第13,137,0,大眼看两融,1
2,1,03-31 02:11,平安银行本周深股通持股市值增加1.89亿元，居银行板块第五,191,1,胖猫周周见,0
3,1,03-29 04:54,平安银行：截至2023年末 本行共有在职员工43119人（含派遣人员、含理财子和分行）,2802,10,平安银行资讯,1
5,1,03-29 08:41,平安银行：融资净偿还5438.84万元，两市排名第16（03-28）,115,0,平安银行资讯,1
...,...,...,...,...,...,...,...
71994,688981,11-14 03:06,华金证券维持中芯国际买入评级 预计2023年净利润同比减少53.08%,260,0,公司研报提示,1
71995,688981,11-14 01:57,光大证券维持中芯国际买入评级 预计2023年净利润同比减少50.33%,192,0,公司研报提示,1
71997,688981,11-14 07:44,华金证券给予中芯国际买入评级，代工龙头营收持续增长，设备提前进场加速产能扩张,2749,19,中芯国际资讯,1
71998,688981,11-14 07:04,【调研快报】中芯国际接待华泰证券等多家机构调研,1870,1,中芯国际资讯,1


In [84]:
concat_list = [succeed_0]
for i in range(4):
chunk = pd.read_csv(f'/Users/wujiewang/Desktop/QF5214/sentiment_output/succeed_{i+1}.csv', index_col = 0)  # 使用pandas的read_csv函数读取CSV文件为DataFrame
concat_list.append(chunk)
output_all = pd.concat(concat_list, ignore_index = True)
output_all.to_csv('/Users/wujiewang/Desktop/QF5214/sentiment_output/news_senti_clas.csv', encoding = 'utf_8_sig')  # 将处理后的数据保存到CSV文件，指定编码为utf_8_sig以兼容中文

In [ ]:
# 要操作的表名
table_name = "ods_news_ef_stock_sen"

In [ ]:
# 登录数据库需要的信息
host='47.***.***.45',
database='qf5214',
user='user',
password='********'

In [ ]:
connection = connect_to_database(host, user, password, database) # 创建数据库连接

In [ ]:
insert_data(connection, table_name, output_all)

## 后续为测试代码

In [73]:
# 从CSV文件导入评论数据，并展示数据的前几行以检查其结构
comments_df = pd.read_csv('/Users/wujiewang/Desktop/QF5214/stock_comments.csv')  # 使用pandas的read_csv函数读取CSV文件为DataFrame
comments_df.head()

,stock_code,time,comments,user_name
0,1,03-29 07:57,平安银行每股派0.719元，股息率6.9%。兴业银行每股派1.040元，股息率6,海兴大吉
1,1,03-27 08:38,平安银行目前股息率最高,价值投资棒棒棒
2,1,03-26 07:13,【炜炜道来】聊聊本轮调整时的操作策略和风险点,边风炜说投资
3,1,03-24 09:08,部分银行年报业绩快报点评,价值投资棒棒棒
4,1,03-23 07:15,人们纠结的还是分红，前几天由于平安银行的正面示范作用，长沙走得还挺好，昨天，中信,轻松的苗飞文


In [79]:
# 截取片段，准备接口测试
temp = comments_df[comments_df['stock_code'] == 166]
temp

,stock_code,time,comments,user_name
2400,166,02-09 01:12,最后一天也不给你好过年，原来如此,所向无敌跟着我
2401,166,02-02 05:19,申万宏源机会北交所改革迎来新进展，920代码号段进入仿真测试阶段，直联审核,jeeptutu
2402,166,01-30 03:04,你了解申万宏源的系统内幕吗？让人心惊肉跳,股友C99x557569
2403,166,01-22 11:14,百股跌停为哪般？相信你所相信的，2730点为自己而战！,铭伟99999
2404,166,01-06 08:04,#宝总初恋恋情#《繁花》中宝总的结局，是A股无数个小散的归宿！阿宝通过炒股票成为,Tgfsrbggtppp
...,...,...,...,...
2795,166,09-07 06:54,第二次破板果断从其他证券全换进申万宏源，从第一创业中抢过龙头，将会是券商后期行情,第七卫队
2796,166,09-06 09:06,因为我用的是语音，错别字挺多的，但是没关系，我也不需要什么关注，只是讲一下心里话,寂寞爱上了股票
2797,166,09-06 09:05,做为申公公的老钉子户，说几句心里话！目前，深万宏源自己股票，他从上市到现在已经调,寂寞爱上了股票
2798,166,09-06 08:35,今天市场全线上涨，收盘创业板指数上涨4.06%，科创50指数上涨2.72%，沪指,账户已注销


In [77]:
# 测试一行文字
print(temp['comments'].iloc[1])
print(aipNlp.sentimentClassify(temp['comments'].iloc[1]))  # 条件判断

申万宏源机会北交所改革迎来新进展，920代码号段进入仿真测试阶段，直联审核
{'text': '申万宏源机会北交所改革迎来新进展，920代码号段进入仿真测试阶段，直联审核', 'items': [{'confidence': 0.913902, 'negative_prob': 0.0387442, 'positive_prob': 0.961256, 'sentiment': 2}], 'log_id': 1774632720225526015}


In [48]:
test = temp['comments'].iloc[:10].apply(lambda x: aipNlp.sentimentClassify(x))  # 条件判断

In [56]:
test[2404]

{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}

👆查找问题所在：访问素嘟过快，QPS 超限

In [63]:
def posi_or_nega(comment):
time.sleep(1)  # 等待一定时间，避免频繁请求API导致访问限制
try:  # 异常处理，尝试执行代码块，处理潜在的错误
posi_prob = (aipNlp.sentimentClassify(comment))['items'][0]['positive_prob']  # 条件判断
nega_prob = (aipNlp.sentimentClassify(comment))['items'][0]['negative_prob']  # 条件判断
if posi_prob > nega_prob:  # 条件判断
return 1
elif posi_prob < nega_prob:  # 条件判断
return (-1)
else:  # 条件判断
return 0
except:  # 异常处理，尝试执行代码块，处理潜在的错误
return 'connection loss'

In [64]:
# 测试一个列表
temp['sentiment'] = 0
temp['sentiment'] = [posi_or_nega(comment) for comment in temp['comments']]

/var/folders/lw/5lp1wqys1c50w56nj8g_n_7w0000gn/T/ipykernel_19502/904195524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['sentiment'] = 0
/var/folders/lw/5lp1wqys1c50w56nj8g_n_7w0000gn/T/ipykernel_19502/904195524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['sentiment'] = [posi_or_nega(comment) for comment in temp['comments']]


In [80]:
temp

,stock_code,time,comments,user_name
2400,166,02-09 01:12,最后一天也不给你好过年，原来如此,所向无敌跟着我
2401,166,02-02 05:19,申万宏源机会北交所改革迎来新进展，920代码号段进入仿真测试阶段，直联审核,jeeptutu
2402,166,01-30 03:04,你了解申万宏源的系统内幕吗？让人心惊肉跳,股友C99x557569
2403,166,01-22 11:14,百股跌停为哪般？相信你所相信的，2730点为自己而战！,铭伟99999
2404,166,01-06 08:04,#宝总初恋恋情#《繁花》中宝总的结局，是A股无数个小散的归宿！阿宝通过炒股票成为,Tgfsrbggtppp
...,...,...,...,...
2795,166,09-07 06:54,第二次破板果断从其他证券全换进申万宏源，从第一创业中抢过龙头，将会是券商后期行情,第七卫队
2796,166,09-06 09:06,因为我用的是语音，错别字挺多的，但是没关系，我也不需要什么关注，只是讲一下心里话,寂寞爱上了股票
2797,166,09-06 09:05,做为申公公的老钉子户，说几句心里话！目前，深万宏源自己股票，他从上市到现在已经调,寂寞爱上了股票
2798,166,09-06 08:35,今天市场全线上涨，收盘创业板指数上涨4.06%，科创50指数上涨2.72%，沪指,账户已注销


In [66]:
temp[temp['sentiment'] == 'connection loss'].count()

stock_code    0
time          0
comments      0
user_name     0
sentiment     0
dtype: int64

In [69]:
temp[temp['sentiment'] == 0]

,stock_code,time,comments,user_name,sentiment


In [ ]:
# 定义一个函数用于分析特定股票代码对应的评论数据的情感倾向，并计算积极与消极情感的比例
def analyze(code):
APP_ID = 'your app id' # 刚才控制台应用界面的appID
API_KEY = 'your api key' # 控制台应用界面的APIKey
SECRET_KEY = 'your secret key' # 控制台应用界面的SecretKey
positive_nums = 0 # 设定正向情感评论数量
nagative_nums = 0 # 设定负向情感评论数量
count = 0 # 总评论数量计数器
aipNlp = AipNlp(APP_ID, API_KEY, SECRET_KEY) # 调用分析工具  # 实例化AipNlp类，使用你的百度云应用的APP_ID, API_KEY和SECRET_KEY
lines = open(str(code) + '.txt').readlines() # 打开爬取的新闻评论文本
for line in lines: # 说明：每行为一个评论，按行分析情感
if not line.isspace():  # 条件判断
line = line.strip()
try:  # 异常处理，尝试执行代码块，处理潜在的错误
result = aipNlp.sentimentClassify(line) # 使用工具包中的sentimentClassify情感分析工具进行分析  # 条件判断
positive_prob = result['items'][0]['positive_prob'] # 记录分析出的“是积极情感”的可能性
nagative_prob = result['items'][0]['negative_prob'] # 记录分析出的“是消极情感”的可能性
count += 1 # 记录分析总数
if positive_prob >= nagative_prob:  # 条件判断
positive_nums += 1
else:  # 条件判断
nagative_nums += 1
except:  # 异常处理，尝试执行代码块，处理潜在的错误
pass
avg_positive = positive_nums / count
avg_nagative = nagative_nums / count
print('股票代码：',code, '消极比例：', avg_nagative, '积极比例：',avg_positive)
return {'股票代码':code, '消极比例':avg_nagative, '积极比例':avg_positive}